# Using Foursquare API - Capital Cities Venues

## Content:

1. Introduction
2. Foursquare API - top venues data collection
3. Data Preprocessing:  
    3.1. Quality validation  
    3.2. Data Collection  
    3.3. One Hot Encoding and Data Normalization  
4. Summary  
5. Refrences

### 1. Introduction:  
* In this notebook we will use the Foursquare API to gather top venues data for each capital city in our *__Capital_Regions.csv__* file.  
* Some preprocessing will be done to insure the quality of our dataframe.
* We will one hot encoding and normalize our data.

In [2]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# The code was removed by Watson Studio for sharing.

### 2. Foursquare API - top venues data collection:

* We will have to create a request URL for each city in our dataframe.  
* Before we do so we will write a function that will create a request for each city in a dataframe.

In [4]:
def get_city_venues (dataframe, radius, LIMIT):
    
    '''
    This function takes a city name, coordinates,
    a search raduius and a limit of searchers for a location.
    Finds top venues (set by the LIMIT) in a locations by
    using the foursquare API.
    '''
    # Select values for venues search: 
    city_name = dataframe['Capital City']
    city_lat = dataframe['City Latitude']
    city_lng = dataframe['City Longitude']
    
    # Empty list for retrived venues of capital city:
    venues_list = [] 
    
    # Define the dataframe's columns:
    venues_columns = ['Capital City', 
                      'City Latitude', 
                      'City Longitude',
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']

    # Find each city's top venues:
    for city, lat, lng in zip(city_name, city_lat, city_lng):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
        .format(CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT
               )

        # GET request to retrive top venues of city:
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # Add the results into venue list:
        for venue in results:
            venues_list.append(
                [
                    city,
                    lat,
                    lng,
                    venue['venue']['name'],
                    venue['venue']['location']['lat'],
                    venue['venue']['location']['lng'],
                    venue['venue']['categories'][0]['name']
                ]
            )
    
    # Create a dataframe of city's venues:
    city_venues = pd.DataFrame(venues_list, columns = venues_columns)

    return(city_venues)

### 3. Data Preprocessing:

#### 3.1. Quality validation:

In [5]:
# Load "Capital_Regions.csv":
capital_raw = pd.read_csv('Capital_Regions.csv')

capital_raw.head()

,Country,Capital City,City Latitude,City Longitude,Region,Income Group
0,Albania,Tirana,41.315886,19.900912,Europe & Central Asia,2
1,Andorra,Andorra la Vella,42.506939,1.521247,Europe & Central Asia,3
2,Antigua and Barbuda,St. John's,17.118457,-61.844851,Latin America & Caribbean,3
3,Argentina,Buenos Aires,-34.607568,-58.437089,Latin America & Caribbean,2
4,Aruba,Oranjestad,12.526874,-70.035685,Latin America & Caribbean,3


In [6]:
capital_raw.shape

(82, 6)

#### 3.1.1. Testing the *__get_city_venues()__* function - Test 1:  
Now we will create a test dataframe to make sure our function works properly.

In [7]:
# Create first test dataframe:
city_venues_raw1 = capital_raw.copy()

We will first run our test finding one venue per city in a 3 kilometer radius.  
Limiting our test to one venue per city will allow us to run it quickly.

In [8]:
# Radius of search set to 3 kilometers:
radius = 3000

# Limit calls per location:
LIMIT = 1

In [10]:
# Run first test:
city_venues_raw1 = get_city_venues(city_venues_raw1, radius, LIMIT)

In [11]:
city_venues_raw1.head()

,Capital City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tirana,41.315886,19.900912,Rexhekri Resort,41.331228,19.905316,Pool
1,Andorra la Vella,42.506939,1.521247,La Borda Pairal,42.505584,1.521477,Restaurant
2,St. John's,17.118457,-61.844851,Hemingways Caribbean Cafe,17.121026,-61.844809,Caribbean Restaurant
3,Buenos Aires,-34.607568,-58.437089,Co-Pain Boulangerie (ex-Franck Dauffouis),-34.607693,-58.438662,Bakery
4,Oranjestad,12.526874,-70.035685,Wonders Boutique Hotel,12.528544,-70.036911,Bed & Breakfast


In [12]:
city_venues_num = city_venues_raw1.shape[0]
capital_raw_num = capital_raw.shape[0]
cities_diff = capital_raw_num - city_venues_num
print(\
'Venues found for {} cities and original dataframe has {} cities, we have {} missing value\s'\
      .format(city_venues_num, capital_raw_num, cities_diff)
     )

Venues found for 81 cities and original dataframe has 82 cities, we have 1 missing value\s


#### 3.1.2. *find_missing()* function:  
To address the issue where we lost data in the process of getting venues' data, we will write a function that finds which city's data was lost in the process.

In [13]:
def find_missing (df_original, df_missing, column):
    
    '''
    Checks if there are missing entries in a new
    dataframe by comparing values with the original
    dataframe checking values in a selcted column.
    
    Takes two dataframes:
    1) Orignal dataframe
    2) Dataframe suspected as missing entries
    
    Returns the index of first unmatched value.
    '''
    
    
    # Iterate through rows of df_original:
    for i in df_original.index:

        # Returns the first unmatched value of original dataframe: 
        if df_original.iloc[i][column] != df_missing.iloc[i][column]:
            print(df_original.iloc[i])
            index = i
            return index
    print('All values match.')
        

Let's use our *__find_missing()__* funciton to find our missing city.

In [14]:
# Find the missing city:
index = find_missing(capital_raw, city_venues_raw1, 'Capital City')

Country                              Brazil
Capital City                       Brasília
City Latitude                      -10.3333
City Longitude                        -53.2
Region            Latin America & Caribbean
Income Group                              2
Name: 12, dtype: object


Through a simple search on the web we can find that *__Brasília__* correct coordinates, whcih are (-15.8267, -47.9218).  
This shows us that we searched the wrong coordinates earlier.  
If there are no venues withing our search radius from the wrong coordinates, then we wont get any results and that city would be dropped from our dataframe.

#### 3.1.4. Applying a manual fix to the missing city coordinates.  

In [15]:
# Create second test dataframe:
city_venues_raw2 = capital_raw.copy()

In [16]:
# Fix Brasília coordinates
correct_coords = [-15.8267, -47.9218]
city_venues_raw2.loc[index, 'City Latitude'] = correct_coords[0]
city_venues_raw2.loc[index, 'City Longitude'] = correct_coords[1]
city_venues_raw2.iloc[index]

Country                              Brazil
Capital City                       Brasília
City Latitude                      -15.8267
City Longitude                     -47.9218
Region            Latin America & Caribbean
Income Group                              2
Name: 12, dtype: object

#### 3.1.4. Testing the *__get_city_venues()__* function - Test 2:  
Validating we have fixed the missing city issue.

In [17]:
# Run second test:
city_venues_raw2 = get_city_venues(city_venues_raw2, radius, LIMIT)

Let's see if our *__find_missing()__* function will give us a message of no missing values.

In [18]:
find_missing(capital_raw, city_venues_raw2, 'Capital City')

All values match.


We can also see that *__Brasília__* is in the second test's results by checking out the dataframe.

In [19]:
city_venues_raw2.head(10)

,Capital City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tirana,41.315886,19.900912,Rexhekri Resort,41.331228,19.905316,Pool
1,Andorra la Vella,42.506939,1.521247,La Borda Pairal,42.505584,1.521477,Restaurant
2,St. John's,17.118457,-61.844851,Hemingways Caribbean Cafe,17.121026,-61.844809,Caribbean Restaurant
3,Buenos Aires,-34.607568,-58.437089,Co-Pain Boulangerie (ex-Franck Dauffouis),-34.607693,-58.438662,Bakery
4,Oranjestad,12.526874,-70.035685,Wonders Boutique Hotel,12.528544,-70.036911,Bed & Breakfast
5,Vienna,48.208354,16.372504,Stephansplatz,48.208299,16.371880,Plaza
6,Bridgetown,13.097783,-59.618418,Subway,13.097955,-59.617446,Sandwich Place
7,Minsk,53.902334,27.561879,Grand Café,53.902347,27.556641,French Restaurant
8,Brussels,50.843671,4.367437,Filigranes,50.843430,4.368396,Bookstore
9,Belmopan,17.250199,-88.770018,Chon Saan,17.255654,-88.763213,Chinese Restaurant


__The missing city issue was fixed and we can continue.__

#### 3.2. Data Collection:  
We will now find the top 100 venues for each city in our dataframe.  
Let's extend our search radius to 5 kilometers and the LIMIT to 100.

In [20]:
# Radius of search set to 3 kilometers
radius = 5000

# Limit calls per location
LIMIT = 100

In [21]:
# Create final dataframe:
top100_city_venues  = city_venues_raw2.copy()

In [22]:
# Collect top 100 venues for each city in "city_venues_final":
top100_city_venues = get_city_venues(top100_city_venues, radius, LIMIT)

In [23]:
# Print a report of the data colleciton trough the Foursquare API:
print(\
'We have retrived {} venues across our {} capital cities.'\
      .format(top100_city_venues.shape[0],len(top100_city_venues['Capital City'].unique()))
     )

We have retrived 7146 venues across our 82 capital cities.


#### 3.3. One Hot Encoding and Data Normalization:  
Last thing we will take care of in this notebook is a *__One Hot Endcoding__* and *__Normalizing__* our data.  
* In *__top100_city_venues__* we have multiple rows per city.
* By using One Hot Encoding each row gets a value of 1 only for the column of its venues type.  
* By aggregating our data using the mean value of each colum for each city we are essentially normalizing each venue type value for each city - meaning each value will be between 0 and 1.

In [24]:
# One hot encoding
capital_onehot = pd.get_dummies(top100_city_venues['Venue Category'], prefix = '', prefix_sep = '')

# Add "Capital City" column back to dataframe
capital_onehot['Capital City'] = top100_city_venues['Capital City'] 

capit_venues_normalized = capital_onehot.groupby('Capital City').mean().reset_index()
capit_venues_normalized.head()

,Capital City,ATM,Accessories Store,Advertising Agency,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cave,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Gym,College Library,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dive Shop,Dive Spot,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kafenio,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Northeastern Brazilian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Event,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outlet Store,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Re

Let's export the *__venues_final_mean__* dataframe as a csv file for future snalysis.

In [25]:
capit_venues_normalized.to_csv(r'Capital_Venues_Normalized.csv', index = False)

### 4. Summary:  

Overall, in this notebook we:
1. We written a function that takes a location dataframe and returns top venues using the Foursquare API.  
2. A test was performed to make sure our top venues function works properly.
3. Data aggregation and normalization was done to the top venues dataframe.

__Thank you for taking your time to read through this!__

### 5. References:

Foursquare. (n.d.). *Foursquare Developer*. Retrieved December 28, 2019, from Foursquare:
https://developer.foursquare.com/